In [54]:
import requests
import socket
import json
import time
import logging
import random
import numpy as np

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s [%(levelname)s] %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

def ConvertToSimTime_us(start_time, time_ratio, day, running_time):
    return (time.time() - start_time - (day - 1) * running_time) * time_ratio

class BotsClass:
    def __init__(self, username, password):
        self.username = username
        self.password = password
    def login(self):
        pass
    def init(self):
        pass
    def bod(self):
        pass
    def work(self):
        pass
    def eod(self):
        pass
    def final(self):
        psss

In [58]:
class BotsDemoClass(BotsClass):
    def __init__(self, username, password):
        super().__init__(username, password);
        self.api = InterfaceClass("https://trading.competition.ubiquant.com")
        self.position = 0
        self.real_pos = 0
        self.activeorder_buy = np.empty((0,3))
        self.activeorder_sell = np.empty((0,3)) # update active order
        self.last_trade_price = 0
    def login(self):
        response = self.api.sendLogin(self.username, self.password)
        if response["status"] == "Success":
            self.token_ub = response["token_ub"]
            logger.info("Login Success: {}".format(self.token_ub))
        else:
            logger.info("Login Error: ", response["status"])
    def GetInstruments(self):
        response = self.api.sendGetInstrumentInfo(self.token_ub)
        if response["status"] == "Success":
            self.instruments = []
            for instrument in response["instruments"]:
                self.instruments.append(instrument["instrument_name"])
            logger.info("Get Instruments: {}".format(self.instruments))
    def init(self):
        response = self.api.sendGetGameInfo(self.token_ub)
        if response["status"] == "Success":
            self.start_time = response["next_game_start_time"]
            self.running_days = response["next_game_running_days"]
            self.running_time = response["next_game_running_time"]
            self.time_ratio = response["next_game_time_ratio"]
        self.GetInstruments()
        self.day = 0
    def bod(self):
        pass        
    def work(self): 
        asset = self.instruments[2]
        LOB = self.api.sendGetLimitOrderBook(self.token_ub, asset)
        trade_info = self.api.sendGetTrade(self.token_ub, asset)
        t = ConvertToSimTime_us(self.start_time, self.time_ratio, self.day, self.running_time)
        # update activeorder
        if (trade_info != None) and (trade_info['trade_list']):
            for order in trade_info['trade_list']:
                if order['order_index'] in self.activeorder_buy:
                    self.position += order['trade_volume']
                    self.last_trade_price = order['trade_price']
                    index = np.where(self.activeorder_buy[:,0] == order['order_index'])
                    if order['remain_volume']==0:
                        self.activeorder_buy = np.delete(self.activeorder_buy,index,axis=0)
                    else:
                        self.activeorder_buy[index,2] = order['remain_volume']
                else:
                    self.position -= order['trade_volume']
                    index = np.where(self.activeorder_sell[:,0] == order['order_index'])
                    if order['remain_volume']==0:
                        self.activeorder_sell = np.delete(self.activeorder_sell,index,axis=0)
                    else:
                        self.activeorder_sell[index,2] = order['remain_volume']
            self.activeorder_buy = self.activeorder_buy[self.activeorder_buy[:,1].argsort()]
            self.activeorder_sell = self.activeorder_sell[self.activeorder_sell[:,1].argsort()]#sort by price, row 0 is minimum
            self.real_pos = self.position - self.activeorder_sell[:,2].sum()
        
        if LOB["status"] == "Success":
            askprice = np.array([float(i) for i in LOB["lob"]["askprice"]])
            bidprice = np.array([float(i) for i in LOB["lob"]["bidprice"]])
            askvol = np.array([float(i) for i in LOB["lob"]["askvolume"]])
            bidvol = np.array([float(i) for i in LOB["lob"]["bidvolume"]])
            mi_2 = (askvol[0]*bidprice[0]+askvol[1]*bidprice[1]+bidvol[0]*askprice[0]+bidvol[1]*askprice[1])/(askvol[0]+askvol[1]+bidvol[0]+bidvol[1])
            if ((askprice[0]-bidprice[0])>0.03): 
                
                response = self.api.sendOrder(self.token_ub, asset, t, "buy", bidprice[0]+0.01 , 100)
                self.activeorder_buy = np.append(self.activeorder_buy, [[response['index'],bidprice[0]+0.01,100]],axis=0)
                
                
            if self.real_pos > 0:
                price = self.last_trade_price+0.03
                response = self.api.sendOrder(self.token_ub, asset, t, "sell", price , 100)
                self.activeorder_sell = np.append(self.activeorder_sell, [[response['index'],price,100]],axis=0)
            
            #stop loss
            if bidprice[0] < self.last_trade_price - 0.02:
                for order in self.activeorder_sell:
                    self.sendCancel(self.token_ub , asset , t , order[0])
                self.api.sendOrder(self.token_ub, asset, t, "buy", bidprice[0] , self.position)
            
    def eod(self):
        pass
    def final(self):
        pass

In [59]:
class InterfaceClass:
    def __init__(self, domain_name):
        self.domain_name = domain_name
        self.session = requests.Session()
    def sendLogin(self, username, password):
        url = self.domain_name + "/api/Login"
        data = {
            "user": username,
            "password": password
        }
        response = self.session.post(url, data=json.dumps(data)).json()
        return response
    
    def sendGetGameInfo(self, token_ub):
        url = self.domain_name + "/api/TradeAPI/GetGAmeInfo"

    def sendOrder(self, token_ub, instrument, localtime, direction, price, volume):
        logger.debug("Order: Instrument: {}, Direction:{}, Price: {}, Volume:{}".format(instrument, direction, price, volume))
        url = self.domain_name + "/api/TradeAPI/Order"
        data = {
            "token_ub": token_ub,
            "user_info": "NULL",
            "instrument": instrument,
            "localtime": localtime,
            "direction": direction,
            "price": price,
            "volume": volume,
        }
        response = self.session.post(url, data=json.dumps(data)).json()
        return response

    def sendCancel(self, token_ub, instrument, localtime, index):
        logger.debug("Cancel: Instrument: {}, index:{}".format(instrument, index))
        url = self.domain_name + "/api/TradeAPI/Cancel"
        data = {
            "token_ub": token_ub,
            "user_info": "NULL",
            "instrument": instrument,
            "localtime": 0,
            "index": index
        }
        response = self.session.post(url, data=json.dumps(data)).json()
        return response

    def sendGetLimitOrderBook(self, token_ub, instrument):
        logger.debug("GetLimitOrderBOok: Instrument: {}".format(instrument))
        url = self.domain_name + "/api/TradeAPI/GetLimitOrderBook"
        data = {
            "token_ub": token_ub,
            "instrument": instrument
        }
        response = self.session.post(url, data=json.dumps(data)).json()
        return response

    def sendGetUserInfo(self, token_ub):
        logger.debug("GetUserInfo: ")
        url = self.domain_name + "/api/TradeAPI/GetUserInfo"
        data = {
            "token_ub": token_ub,
        }
        response = self.session.post(url, data=json.dumps(data)).json()
        return response

    def sendGetGameInfo(self, token_ub):
        logger.debug("GetGameInfo: ")
        url = self.domain_name + "/api/TradeAPI/GetGameInfo"
        data = {
            "token_ub": token_ub,
        }
        response = self.session.post(url, data=json.dumps(data)).json()
        return response

    def sendGetInstrumentInfo(self, token_ub):
        logger.debug("GetInstrumentInfo: ")
        url = self.domain_name + "/api/TradeAPI/GetInstrumentInfo"
        data = {
            "token_ub": token_ub,
        }
        response = self.session.post(url, data=json.dumps(data)).json()
        return response

    def sendGetTrade(self, token_ub, instrument):
        logger.debug("GetTrade: Instrment: {}".format(instrument))
        url = self.domain_name + "/api/TradeAPI/GetTrade"
        data = {
            "token_ub": token_ub,
            "instrument_name": instrument
        }
        response = self.session.post(url, data=json.dumps(data)).json()
        return response

    def sendGetActiveOrder(self, token_ub):
        logger.debug("GetActiveOrder: ")
        url = self.domain_name + "/api/TradeAPI/GetActiveOrder"
        data = {
            "token_ub": token_ub,
        }
        response = self.session.post(url, data=json.dumps(data)).json()
        return response

In [60]:
bot = BotsDemoClass('UBIQ_TEAM359', "eeKvoJiLv")
bot.login()
bot.init()
SimTimeLen = 14400
endWaitTime = 300
while True:
    if ConvertToSimTime_us(bot.start_time, bot.time_ratio, bot.day, bot.running_time) < SimTimeLen:
        break
    else:
        bot.day += 1

while bot.day <= bot.running_days:
    while True:
        if ConvertToSimTime_us(bot.start_time, bot.time_ratio, bot.day, bot.running_time) > -900:
            break
    bot.bod()
    now = round(ConvertToSimTime_us(bot.start_time, bot.time_ratio, bot.day, bot.running_time))
    for s in range(now, SimTimeLen + endWaitTime):
        while True:
            if ConvertToSimTime_us(bot.start_time, bot.time_ratio, bot.day, bot.running_time) >= s:
                break
        t = ConvertToSimTime_us(bot.start_time, bot.time_ratio, bot.day, bot.running_time)
        logger.info("Work Time: {}".format(t))
        if t < SimTimeLen - 30:
            bot.work()
    bot.eod()
    bod.day += 1
bot.final()

2023-12-04 11:50:16,826 [INFO] Login Success: UBIQ_TEAM359_eeKvoJiLv
2023-12-04 11:50:16,826 [INFO] Login Success: UBIQ_TEAM359_eeKvoJiLv
2023-12-04 11:50:16,826 [INFO] Login Success: UBIQ_TEAM359_eeKvoJiLv
2023-12-04 11:50:16,826 [INFO] Login Success: UBIQ_TEAM359_eeKvoJiLv
2023-12-04 11:50:16,830 [DEBUG] GetGameInfo: 
2023-12-04 11:50:16,830 [DEBUG] GetGameInfo: 
2023-12-04 11:50:16,830 [DEBUG] GetGameInfo: 
2023-12-04 11:50:16,830 [DEBUG] GetGameInfo: 
2023-12-04 11:50:17,084 [DEBUG] GetInstrumentInfo: 
2023-12-04 11:50:17,084 [DEBUG] GetInstrumentInfo: 
2023-12-04 11:50:17,084 [DEBUG] GetInstrumentInfo: 
2023-12-04 11:50:17,084 [DEBUG] GetInstrumentInfo: 
2023-12-04 11:50:17,149 [INFO] Get Instruments: ['UBIQ000', 'UBIQ001', 'UBIQ002', 'UBIQ003', 'UBIQ004', 'UBIQ005', 'UBIQ006', 'UBIQ007', 'UBIQ008', 'UBIQ009', 'UBIQ010', 'UBIQ011', 'UBIQ012', 'UBIQ013', 'UBIQ014', 'UBIQ015', 'UBIQ016', 'UBIQ017', 'UBIQ018', 'UBIQ019', 'UBIQ020', 'UBIQ021', 'UBIQ022', 'UBIQ023', 'UBIQ024', 'UBIQ025

2023-12-04 11:50:18,713 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:18,713 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:18,713 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:18,758 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:18,758 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:18,758 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:18,758 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:18,832 [INFO] Work Time: 12188.324768543243
2023-12-04 11:50:18,832 [INFO] Work Time: 12188.324768543243
2023-12-04 11:50:18,832 [INFO] Work Time: 12188.324768543243
2023-12-04 11:50:18,832 [INFO] Work Time: 12188.324768543243
2023-12-04 11:50:18,834 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:18,834 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:18,834 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:18,834 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:5

2023-12-04 11:50:20,621 [INFO] Work Time: 12206.210298538208
2023-12-04 11:50:20,621 [INFO] Work Time: 12206.210298538208
2023-12-04 11:50:20,621 [INFO] Work Time: 12206.210298538208
2023-12-04 11:50:20,623 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:20,623 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:20,623 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:20,623 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:20,686 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:20,686 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:20,686 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:20,686 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:20,809 [INFO] Work Time: 12208.098821640015
2023-12-04 11:50:20,809 [INFO] Work Time: 12208.098821640015
2023-12-04 11:50:20,809 [INFO] Work Time: 12208.098821640015
2023-12-04 11:50:20,809 [INFO] Work Time: 12208.098821640015
2023-12-04 11:50:20,812 [DEBUG] GetLimitOrder

2023-12-04 11:50:22,512 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:22,512 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:22,512 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:22,582 [INFO] Work Time: 12225.826199054718
2023-12-04 11:50:22,582 [INFO] Work Time: 12225.826199054718
2023-12-04 11:50:22,582 [INFO] Work Time: 12225.826199054718
2023-12-04 11:50:22,582 [INFO] Work Time: 12225.826199054718
2023-12-04 11:50:22,585 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:22,585 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:22,585 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:22,585 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:22,637 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:22,637 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:22,637 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:22,637 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:22,702 [INFO] Work Time: 122

2023-12-04 11:50:24,674 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:24,674 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:24,674 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:24,723 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:24,723 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:24,723 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:24,723 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:24,781 [INFO] Work Time: 12247.818870544434
2023-12-04 11:50:24,781 [INFO] Work Time: 12247.818870544434
2023-12-04 11:50:24,781 [INFO] Work Time: 12247.818870544434
2023-12-04 11:50:24,781 [INFO] Work Time: 12247.818870544434
2023-12-04 11:50:24,784 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:24,784 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:24,784 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:24,784 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:5

...

2023-12-04 11:50:42,665 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:42,714 [INFO] Work Time: 12427.149140834808
2023-12-04 11:50:42,714 [INFO] Work Time: 12427.149140834808
2023-12-04 11:50:42,714 [INFO] Work Time: 12427.149140834808
2023-12-04 11:50:42,714 [INFO] Work Time: 12427.149140834808
2023-12-04 11:50:42,718 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:42,718 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:42,718 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:42,718 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:42,826 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:42,826 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:42,826 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:42,826 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:42,959 [INFO] Work Time: 12429.595038890839
2023-12-04 11:50:42,959 [INFO] Work Time: 12429.595038890839
2023-12-04 11:50:42,959 [INFO] Work Time: 124

2023-12-04 11:50:44,286 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:44,533 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:44,533 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:44,533 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:44,533 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:44,581 [INFO] Work Time: 12445.81279039383
2023-12-04 11:50:44,581 [INFO] Work Time: 12445.81279039383
2023-12-04 11:50:44,581 [INFO] Work Time: 12445.81279039383
2023-12-04 11:50:44,581 [INFO] Work Time: 12445.81279039383
2023-12-04 11:50:44,583 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:44,583 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:44,583 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:44,583 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:44,637 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:44,637 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:44,637 [DEBUG] GetTrad

2023-12-04 11:50:46,235 [INFO] Work Time: 12462.358219623566
2023-12-04 11:50:46,238 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:46,238 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:46,238 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:46,238 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:46,299 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:46,299 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:46,299 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:46,299 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:46,377 [INFO] Work Time: 12463.77053976059
2023-12-04 11:50:46,377 [INFO] Work Time: 12463.77053976059
2023-12-04 11:50:46,377 [INFO] Work Time: 12463.77053976059
2023-12-04 11:50:46,377 [INFO] Work Time: 12463.77053976059
2023-12-04 11:50:46,381 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:46,381 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:46,381 [DEBU

2023-12-04 11:50:47,869 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:47,913 [INFO] Work Time: 12479.134888648987
2023-12-04 11:50:47,913 [INFO] Work Time: 12479.134888648987
2023-12-04 11:50:47,913 [INFO] Work Time: 12479.134888648987
2023-12-04 11:50:47,913 [INFO] Work Time: 12479.134888648987
2023-12-04 11:50:47,918 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:47,918 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:47,918 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:47,918 [DEBUG] GetLimitOrderBOok: Instrument: UBIQ002
2023-12-04 11:50:47,961 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:47,961 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:47,961 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:47,961 [DEBUG] GetTrade: Instrment: UBIQ002
2023-12-04 11:50:48,394 [INFO] Work Time: 12483.94821882248
2023-12-04 11:50:48,394 [INFO] Work Time: 12483.94821882248
2023-12-04 11:50:48,394 [INFO] Work Time: 12483

AttributeError: 'BotsDemoClass' object has no attribute 'sendCancel'